In [255]:
from pydoc import describe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.impute import SimpleImputer
from sklearn import preprocessing

In [256]:
#Lecture des données
data = pd.read_excel("/home/congo/Bureau/2022-m1-staps/data_motives/data_motives_final.xlsx","haller",) 

In [257]:
# mise a zero des valeurs manquantes
data = data.fillna(0)

In [258]:
data = data.to_numpy()
data = data[:,39:]
nlin ,ncol =data.shape
print('data' ,data.shape)

data (1075, 71)


Préprosseccing

In [259]:
# rendre toute les valeurs positives
data_correct = data
data = np.absolute(data)

In [260]:
#Supression des valeurs abérantes par rapport au seuil
seuil = 5*np.std(data,axis=0,dtype = np.float64)
#print(seuil)
data = np.array(data)
data_tmp = np.zeros_like(data)
data_tmp[data < seuil] = data[data < seuil]
data = data_tmp
#print(data)

In [261]:
#normalisation par ligne (avec ecartype nul)
mean = np.mean(data,axis = 1,dtype = np.float64)
std = np.std(data,axis = 1,dtype = np.float64)

# question : doit -t-on supprimer les personnes y'ayant un écartype nul ? oui 

#print("data shape before normalisation",data.shape)
indices = [i for i, e in enumerate(std) if e == 0]
#print(indices)

data_del = np.delete(data,indices,axis = 0)
data = data_del

#print("data shape after normalisation",data_del.shape)


#normalisation par ligne sans ecartype nul
mean = np.mean(data,axis = 1,dtype = np.float64)
std = np.std(data,axis = 1,dtype = np.float64)
data = (data -mean[:,np.newaxis])/std[:,np.newaxis]


In [262]:
# retour des valeurs négatives

#suppression des lignes à ecartype nul
data_del = np.delete(data_correct,indices,axis = 0)
data_correct = data_del

data_correct = np.array(data_correct)
data_tmp = np.zeros_like(data_correct)
data_tmp[data_correct < 0] = data[data_correct < 0]
data = data_tmp


 Cluster Algorithm 

1) CAH algorithm 

data_train,data_test,data_validation

In [263]:
data_train = data[:int(nlin*0.70),:int(ncol*0.70)]
data_test = data[int(nlin*0.70):,int(ncol*0.70):]

In [264]:

#pour construire un pre-processsing

class MyStandardScaler:
       
    def fit(self,X_train):
        self.mean_=np.mean(X_train,axis=0)
        self.std_=np.std(X_train,axis=0)
                
    def transform(self,X):
        """
        Cette méthode peut notamment s'appliquer aux données test. 
        Mais dans ce cas là, c'est le la moyenne et l'écart type des données train qui doit être utiliser. 
        """
        assert len(X.shape)==2, "X must be a matrix"
        
        "res[i,j]=(X[i,j] - mean[j])/std[j]"
        res=(X-self.mean_[np.newaxis,:])/self.std_[np.newaxis,:]
        return res

    
    def fit_transform(self,X_train):
        self.fit(X_train)
        return self.transform(X_train)